In [1]:
import torch
import numpy as np
from datasets import load_dataset
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

/home/arjun/Desktop/AI_ENV/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
batch_size = 64

# Creating vocabulory

In [3]:
n = 3
embedding_dims = 10 # how many dimensional vector should represent each word in the vocabulory

In [4]:
ds = load_dataset("MuskumPillerum/General-Knowledge")
df = ds['train'][:10]

In [5]:
def tokenize(sentence):
    data = sentence
    split_terms = [',', '.', '!', '?', '(', ')', '&', '$', '+', '-', '/', '*', ';', ':']
    for split_term in split_terms:
        if split_term in sentence:
            data = data.replace(split_term, f' {split_term} ')
    data = data.split()
    return data

tokenize('what is your !! ! name ?    ')

['what', 'is', 'your', '!', '!', '!', 'name', '?']

In [6]:
vocab_list = set()
X = []
for x,y in zip(df['Question'], df['Answer']):
    data = f'Question: {x} Answer: {y}'
    data = data.lower().replace('\\n', '')
    vocab_list.update(tokenize(data))
    X.append(data)

In [7]:
vocab_list.add('<UNK>')

In [8]:
vocab = {v:i for v,i in zip(vocab_list, range(0, len(vocab_list)+1))}
len(vocab)

141

In [9]:
vocab

{'refers': 0,
 'would': 1,
 'or': 2,
 'systems': 3,
 'such': 4,
 'over': 5,
 'tasks': 6,
 'system': 7,
 'narrow': 8,
 'speech': 9,
 'time': 10,
 'interpret': 11,
 'intelligence': 12,
 'translation': 13,
 'decisions': 14,
 'relationships': 15,
 'ai': 16,
 'is': 17,
 'way': 18,
 'analyze': 19,
 'in': 20,
 'various': 21,
 'examples': 22,
 'without': 23,
 'as': 24,
 'algorithms': 25,
 'consists': 26,
 'inspired': 27,
 'generate': 28,
 'network': 29,
 'penalty': 30,
 'of': 31,
 'takes': 32,
 'form': 33,
 'this': 34,
 'on': 35,
 'environment': 36,
 'structure': 37,
 'while': 38,
 'neural': 39,
 'learns': 40,
 'recognition': 41,
 'aims': 42,
 'uses': 43,
 'are': 44,
 'development': 45,
 'feedback': 46,
 'improve': 47,
 'patterns': 48,
 'decision': 49,
 'learning': 50,
 'type': 51,
 'typically': 52,
 'specific': 53,
 'given': 54,
 'predictions': 55,
 'brain': 56,
 '?': 57,
 'based': 58,
 'many': 59,
 'actions': 60,
 'answer': 61,
 'require': 62,
 'being': 63,
 'what': 64,
 'question': 65,
 'na

In [10]:
embedding_matrix = torch.randn(len(vocab),embedding_dims)
embedding_matrix.shape

torch.Size([141, 10])

In [11]:
embedding_matrix[[1,2,3]]

tensor([[ 0.2216,  0.4740, -1.3423, -0.0492,  0.3979, -1.8105, -1.1462,  0.6569,
         -0.1906, -1.9920],
        [ 0.1674, -0.5237, -0.1121, -0.8484,  1.4877, -1.1169, -0.1958,  0.2541,
          0.3205,  0.5984],
        [-0.5154,  1.9534,  0.6767, -1.6047, -0.7326, -0.3458, -1.6799, -1.0070,
          0.8391,  0.0948]])

In [12]:
def get_word_embedding(word, vocab=vocab, embedding_matrix=embedding_matrix):
    if word not in vocab:
        word = '<UNK>'
    embedding = embedding_matrix[vocab[word]]
    return embedding

get_word_embedding('as')

tensor([-0.4289,  0.8801,  0.1586,  0.6226,  0.7116, -0.3868,  0.2205, -0.1981,
         1.0451,  0.7080])

# Creating the model

In [13]:
from print_color import print

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self, n=n, num_hidden_layer=100, vocab_len = len(vocab_list), dim_embedding=10):
        super(NeuralNetwork, self).__init__()
        self.n = n
        self.dim_embedding = dim_embedding
        self.embedding = torch.rand(vocab_len, dim_embedding).to(device)
        self.hidden_layer = nn.Linear((n-1)*dim_embedding, num_hidden_layer)
        self.relu = nn.ReLU()
        self.output = nn.Linear(num_hidden_layer, vocab_len)
        self.softmax = nn.Softmax(dim=1)

    def forward(self,x):
        # x will be the indeices of embedding representing the input words
        # print(self.embedding[x], color='g')
        x_embeddings = self.embedding[x].view(-1,(self.n-1)*self.dim_embedding).to(device)
        # print('banana', color='y')
        # print(x_embeddings.shape, color='g')
        out = self.hidden_layer(x_embeddings)
        out = self.relu(out)
        out = self.output(out)
        # print(out.shape, color='r')
        out = self.softmax(out)
        return out

model = NeuralNetwork(n=3).to(device)

# Creating dataset

In [15]:
dataset = []
for x,y in zip(df['Question'], df['Answer']):
    data = f'Question: {x} Answer: {y}'.lower()
    tokenized_data = tokenize(data)
    for i in range(len(tokenized_data)-n):
        # print(i)
        data_i = tokenized_data[i:i+n]
        dataset.append([vocab[i] if i in vocab else vocab['<UNK>'] for i in data_i])
    # print()

dataset_np = np.array(dataset)
dataset_np.shape

(419, 3)

In [16]:
class QuestionAnswerDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.x = dataset[:,[i for i in range(n-1)]]
        self.y = dataset[:,-1]
        self.m, self.n = self.x.shape
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.m
    
dataset = QuestionAnswerDataset(dataset=dataset_np)
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# Training

In [17]:
num_epochs = 100
lr = 0.0001
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

cuda


In [18]:
# model = torch.compile(model)

In [19]:
lossCategory = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=lr)

In [20]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [23]:
for epoch in range(num_epochs):
    for i, (input, label) in enumerate(dataloader):
        x = input.to(device)
        y = label.to(device)
        # print(x,y)
        # print(model(x))
        # print('cow', color='r')
        # print(model(x).shape)
        # y_pred = torch.argmax(model(x), dim=1)
        # print(y_pred, color='red')
        loss = lossCategory(model(x), y)
        print(loss)
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()


tensor(4.9032, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.8983, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9194, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9148, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9043, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9376, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9173, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9279, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9021, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9204, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.8837, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9127, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9281, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9185, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9186, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9123, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9083, device='cuda:0', grad_fn=